In [32]:
# This is a python script that parses a user provided csv file with task attributes and dependency information
# This file is refered to as the WBS (Work Breakdown Structure)
# The script will build a dictionary from the WBS file and create utility functions to access the data and calculate time estimates 

import pandas as pd

WBS_FILE_PATH = 'wbs_data.csv'  # Path to the WBS file

def load_wbs(file_path):
    """
    Load the Work Breakdown Structure (WBS) from an Excel file.
    
    Args:
        file_path (str): Path to the Excel file containing the WBS.
        
    Returns:
        pd.DataFrame: DataFrame containing the WBS data.
    """
    try:
        wbs_df = pd.read_csv(file_path)

        # Print some information about the loaded DataFrame
        print(f"WBS loaded successfully with {len(wbs_df)} rows and {len(wbs_df.columns)} columns.")
        print("Columns in WBS:", wbs_df.columns.tolist())
        # Optionally, display the first few rows of the DataFrame
        print("First few rows of WBS:")
        print(wbs_df.head())

        return wbs_df
    except Exception as e:
        print(f"Error loading WBS file: {e}")
        return None

df = load_wbs(WBS_FILE_PATH)
    
if df is None:
    print("Failed to load WBS data.")
    df = pd.DataFrame()  # Create an empty DataFrame if loading fails

WBS loaded successfully with 561 rows and 27 columns.
Columns in WBS: ['Code, 1', 'Code, 2', 'Code, 3', 'Code, 4', 'Code, 5', 'Code, 6', 'Description', 'Unit', 'Units', 'Consumption', 'Waste Factor', 'Unit Cost', 'Source Qty', 'Unnamed: 13', 'Tag-Uniformat 1', 'Tag-Uniformat 2', 'Tag-Uniformat 3', 'Tag-Uniformat 4', 'Tag-CSI 1', 'Tag-CSI 2', 'Tag-Cost Type', 'Tag-Labor Resource', 'Tag-CSI Resource 1', 'Tag-CSI Resource 2', 'Unnamed: 24', 'Node', 'Unnamed: 26']
First few rows of WBS:
  Code, 1 Code, 2 Code, 3    Code, 4         Code, 5  Code, 6  \
0       G     NaN     NaN        NaN             NaN      NaN   
1     NaN     G10     NaN        NaN             NaN      NaN   
2     NaN     NaN     NaN  G1031.001             NaN      NaN   
3     NaN     NaN     NaN        NaN  31.23.00.100.0      NaN   
4     NaN     NaN     NaN        NaN             NaN  LEXC001   

                        Description Unit  Units  Consumption  ...  \
0               BUILDING SITEWORK      -      -     

In [33]:
# Drop unnecessary columns
columns_to_drop = [
    'Code, 1', 'Code, 2', 'Code, 3', 'Code, 4', 'Code, 5', 'Tag-Labor Resource', 
    'Tag-CSI Resource 1', 'Tag-CSI Resource 2', 'Unnamed: 24', 'Node', 'Unnamed: 26', 'Unnamed: 13'
]

df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

print("Columns after dropping unnecessary ones:", df.columns.tolist())

Columns after dropping unnecessary ones: ['Code, 6', 'Description', 'Unit', 'Units', 'Consumption', 'Waste Factor', 'Unit Cost', 'Source Qty', 'Tag-Uniformat 1', 'Tag-Uniformat 2', 'Tag-Uniformat 3', 'Tag-Uniformat 4', 'Tag-CSI 1', 'Tag-CSI 2', 'Tag-Cost Type']


In [34]:
# Rename 'Code, 6' column to 'task_code'
df.rename(columns={'Code, 6': 'task_code'}, inplace=True)

In [35]:
# Reorder columns to have 'Tag' columns first
tag_columns = [col for col in df.columns if col.startswith('Tag-')]
other_columns = [col for col in df.columns if not col.startswith('Tag-')]
df = df[tag_columns + other_columns]
print("Columns after reordering:", df.columns.tolist())
print()
print("DataFrame after processing:")
print(df.head())

Columns after reordering: ['Tag-Uniformat 1', 'Tag-Uniformat 2', 'Tag-Uniformat 3', 'Tag-Uniformat 4', 'Tag-CSI 1', 'Tag-CSI 2', 'Tag-Cost Type', 'task_code', 'Description', 'Unit', 'Units', 'Consumption', 'Waste Factor', 'Unit Cost', 'Source Qty']

DataFrame after processing:
           Tag-Uniformat 1           Tag-Uniformat 2  \
0  G - BUILDING SITEWORK                         NaN   
1  G - BUILDING SITEWORK    G10 - Site Preparation     
2  G - BUILDING SITEWORK    G10 - Site Preparation     
3  G - BUILDING SITEWORK    G10 - Site Preparation     
4  G - BUILDING SITEWORK    G10 - Site Preparation     

            Tag-Uniformat 3                    Tag-Uniformat 4  \
0                       NaN                                NaN   
1                       NaN                                NaN   
2  G1030 - Site Earthwork    G1031 - Site Grading Excavation     
3  G1030 - Site Earthwork    G1031 - Site Grading Excavation     
4  G1030 - Site Earthwork    G1031 - Site Grading Excav

In [36]:
# Drop rows with Null values in 'task_code'
df.dropna(subset=['task_code'], inplace=True)

# Drop rows where the Unit is not 'HR', because we are only interested in tasks measured in hours
df = df[df['Unit'] == 'HR']

# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

In [37]:
# For debugging, save the processed DataFrame to a new CSV file
df.to_csv('processed_wbs_data.csv', index=False)

In [ ]:
# Define a function to calculate the time estimate for each task based on an input
def calculate_time_estimate(task_code, input_value, verbose=True) -> float:
    """
    Calculate the time estimate for a given task based on the input value.
    
    Args:
        task_code (str): The code of the task.
        input_value (float): The input value to calculate the time estimate.
        
    Returns:
        float: The calculated time estimate for the task.
    """
    task_row = df[df['task_code'] == task_code]
    
    if task_row.empty:
        print(f"Task code '{task_code}' not found in the WBS.")
        return None
    
    # If there are multiple rows for the same task code, we can sum the time estimates
    # We can assume that the 'Consumption' column contains the time estimates in hours/unit
    consumption = task_row['Consumption'].sum()

    # Calculate the time estimate based on the input value
    time_estimate = consumption * input_value
    if verbose:
        print(f"Task code: {task_code}, Consumption: {consumption}, Input value: {input_value}")
    return time_estimate
